In [2]:
import pandas as pd
import tensorflow as tf
tf.random.set_seed(101)
from keras_preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import roc_auc_score, accuracy_score

In [3]:
print('hello world')

hello world


In [11]:
data = pd.read_csv('/Users/polinanazarova/Desktop/hack_10_2024/train_data/defects.csv', delimiter=';')

In [12]:
data.head()

,main_class,additional_class,filename,x_left_bottom,y_left_bottom,length,width
0,царапины,Царапина на задней панели,2023-12-26 15-38-08.jpg,965.0,3075.0,637.0,1405.0
1,царапины,Царапина на задней панели,2024-01-15 13-40-28.jpg,885.0,1987.0,345.0,270.0
2,царапины,Царапина на задней панели,2024-01-15 13-42-01.jpg,989.0,2664.0,291.0,549.0
3,царапины,Царапина на задней панели,2024-01-15 13-50-51.jpg,1497.0,2797.0,345.0,489.0
4,царапины,Царапина на задней панели,2024-01-15 14-34-26.jpg,1145.0,2844.0,403.0,589.0


In [14]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=(0.8,1),
        horizontal_flip=True,
        vertical_flip=True
        )
val_datagen = ImageDataGenerator(
        rescale=1./255
        )

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
data['main_class'].isna().sum()

104

In [31]:
data['main_class'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
187     True
188     True
189     True
190     True
191     True
Name: main_class, Length: 192, dtype: bool

In [30]:
len(data['main_class'])

192

In [40]:
set(data['main_class'])

{nan,
 'битые пиксели',
 'замок',
 'отсутствует шуруп',
 'проблемы с клавишами',
 'сколы',
 'царапины'}

In [44]:
labels = ['битые пиксели',
          'замок',
          'отсутствует шуруп',
          'проблемы с клавишами',
          'сколы',
          'царапины']

In [59]:
labeled_data = []

for i in range(len(data['main_class'])):
    if not data['main_class'].isna()[i]:
        labeled_data.append((data['main_class'][i], data['filename'][i]))

X = pd.DataFrame(labeled_data, columns=['class', 'filename'])

X.head()

,class,filename
0,царапины,2023-12-26 15-38-08.jpg
1,царапины,2024-01-15 13-40-28.jpg
2,царапины,2024-01-15 13-42-01.jpg
3,царапины,2024-01-15 13-50-51.jpg
4,царапины,2024-01-15 14-34-26.jpg


In [60]:
train, test, _, _ = train_test_split(X, X, test_size=0.1, random_state=42)

In [61]:
len(labeled_data), len(train), len(test)

(88, 79, 9)

dataframe: The source dataframe (in our case, the train and val dataframes created earlier)

directory: The location of our images relative to the current working directory

x_col: The column name in our dataframe where the image filenames are stored

y_col: The column name in our dataframe where the target is stored

target_size: What dimensions we'd like our images resizing to batch_size

class_mode: Our task is binary classification and so we can use the binary mode

In [64]:
IMG_DIM = 224
BATCH_SIZE = 64

train_generator = train_datagen.flow_from_dataframe(train, 
                                                    directory='/Users/polinanazarova/Desktop/hack_10_2024/train_data/data', 
                                                    x_col='filename', 
                                                    y_col='class',
                                                    target_size=(IMG_DIM , IMG_DIM ),
                                                    batch_size=BATCH_SIZE ,
                                                    class_mode='sparse')
val_generator = val_datagen.flow_from_dataframe(test, 
                                                directory='/Users/polinanazarova/Desktop/hack_10_2024/train_data/data', 
                                                x_col='filename', 
                                                y_col='class',
                                                shuffle=False,
                                                target_size=(IMG_DIM ,IMG_DIM ),
                                                batch_size=BATCH_SIZE ,
                                                class_mode='sparse')

Found 79 validated image filenames belonging to 6 classes.
Found 9 validated image filenames belonging to 4 classes.


In [84]:
base_model = EfficientNetB0(input_shape=(IMG_DIM , IMG_DIM ,3),
                                             include_top=False)

base_model.trainable = False

model = Sequential(
        [base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'), 
        Dense(len(labels), activation='sigmoid')]
        )

In [85]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = 'mse',
              metrics='accuracy')

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

history = model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=5)
 

Epoch 1/5
1/1 [==============================] - 9s 9s/step - loss: 5.8660 - accuracy: 0.1250
Epoch 2/5
1/1 [==============================] - 2s 2s/step - loss: 6.2118 - accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 5s 5s/step - loss: 4.2763 - accuracy: 0.1094
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 5.0523 - accuracy: 0.1333
Epoch 5/5
1/1 [==============================] - 5s 5s/step - loss: 4.3801 - accuracy: 0.2031


In [87]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

#load the image
my_image = load_img('/Users/polinanazarova/Desktop/hack_10_2024/train_data/data/2023-12-26 15-16-46.jpg', target_size=(IMG_DIM, IMG_DIM))

#preprocess the image
my_image = img_to_array(my_image)
my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
my_image = preprocess_input(my_image)

#make the prediction
prediction = model.predict(my_image)

1/1 [==============================] - 0s 38ms/step


In [90]:
labels[np.argmax([np.round(x) for x in prediction])]

'проблемы с клавишами'